In [1]:
import pandas as pd
import tensorflow as tf
import os
import shutil
import numpy as np
from tqdm.auto import tqdm

2023-08-14 20:20:02.202940: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 20:20:02.694164: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
DATASET_DIR = "../data/asl-fingerspelling"
TRAIN_LANDMARKS_PATH = DATASET_DIR+os.sep+"train_landmarks"
SUPPLEMENTAL_LANDMARKS_PATH = DATASET_DIR+os.sep+"supplemental_landmarks"
TRAIN_PATH = DATASET_DIR+os.sep+"train.csv"
TFRECORDS_PATH = DATASET_DIR+os.sep+"tfrecords"

In [3]:
train_df = pd.read_csv(TRAIN_PATH)

In [4]:
train_df.head()

,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road


In [5]:
# how many do we have

train_df.shape

(67208, 5)

In [6]:
# how many phrases do we have?

train_df.phrase.nunique()

46478

# analisys

The problem consist in taking as input the sequence of differents hand landmarks and generating text base of that sequence.

Each landmark sequence has it's own parquet

Let's see the first one train_landmarks/5414471.parquet

In [7]:
landmark_sequence = pd.read_parquet(DATASET_DIR+os.sep+"train_landmarks/5414471.parquet").reset_index()

In [8]:
landmark_sequence

,sequence_id,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
0,1816796431,0,0.710588,0.699951,0.705657,0.691768,0.699669,0.701980,0.709724,0.610405,...,-0.245855,-0.269148,-0.129743,-0.251501,-0.278687,-0.266530,-0.152852,-0.257519,-0.275822,-0.266876
1,1816796431,1,0.709525,0.697582,0.703713,0.691016,0.697576,0.700467,0.709796,0.616540,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1816796431,2,0.711059,0.700858,0.706272,0.693285,0.700825,0.703319,0.711549,0.615606,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1816796431,3,0.712799,0.702518,0.707840,0.694899,0.702445,0.704794,0.712483,0.625044,...,-0.370770,-0.408097,-0.185217,-0.325494,-0.343373,-0.328294,-0.203126,-0.315719,-0.326104,-0.314282
4,1816796431,4,0.712349,0.705451,0.709918,0.696006,0.705180,0.706928,0.712685,0.614356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162694,1848182207,296,0.657136,0.635888,0.643259,0.619031,0.633084,0.631827,0.630708,0.533120,...,-0.143147,-0.139659,-0.066276,-0.130910,-0.127341,-0.106674,-0.083439,-0.124994,-0.119394,-0.101404
162695,1848182207,297,0.655706,0.635570,0.642730,0.618637,0.632830,0.631554,0.630344,0.531868,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162696,1848182207,298,0.653681,0.636057,0.643054,0.618643,0.633258,0.631800,0.630059,0.531178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162697,1848182207,299,0.654293,0.635543,0.642558,0.617969,0.632699,0.631167,0.629263,0.531019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# I'm only interested in the hand coordinates, x and y

def get_columns(df):
    columns_right_hand = [column for column in df.columns if "x_right_hand" in column or "y_right_hand" in column]
    columns_left_hand = [column for column in df.columns if "x_left_hand" in column or "y_left_hand" in column]
    return columns_right_hand + columns_left_hand

In [10]:
hand_columns = get_columns(landmark_sequence)

In [11]:
hand_columns

['x_right_hand_0',
 'x_right_hand_1',
 'x_right_hand_2',
 'x_right_hand_3',
 'x_right_hand_4',
 'x_right_hand_5',
 'x_right_hand_6',
 'x_right_hand_7',
 'x_right_hand_8',
 'x_right_hand_9',
 'x_right_hand_10',
 'x_right_hand_11',
 'x_right_hand_12',
 'x_right_hand_13',
 'x_right_hand_14',
 'x_right_hand_15',
 'x_right_hand_16',
 'x_right_hand_17',
 'x_right_hand_18',
 'x_right_hand_19',
 'x_right_hand_20',
 'y_right_hand_0',
 'y_right_hand_1',
 'y_right_hand_2',
 'y_right_hand_3',
 'y_right_hand_4',
 'y_right_hand_5',
 'y_right_hand_6',
 'y_right_hand_7',
 'y_right_hand_8',
 'y_right_hand_9',
 'y_right_hand_10',
 'y_right_hand_11',
 'y_right_hand_12',
 'y_right_hand_13',
 'y_right_hand_14',
 'y_right_hand_15',
 'y_right_hand_16',
 'y_right_hand_17',
 'y_right_hand_18',
 'y_right_hand_19',
 'y_right_hand_20',
 'x_left_hand_0',
 'x_left_hand_1',
 'x_left_hand_2',
 'x_left_hand_3',
 'x_left_hand_4',
 'x_left_hand_5',
 'x_left_hand_6',
 'x_left_hand_7',
 'x_left_hand_8',
 'x_left_hand_9',


In [12]:
landmark_sequence.head()

,sequence_id,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
0,1816796431,0,0.710588,0.699951,0.705657,0.691768,0.699669,0.701980,0.709724,0.610405,...,-0.245855,-0.269148,-0.129743,-0.251501,-0.278687,-0.266530,-0.152852,-0.257519,-0.275822,-0.266876
1,1816796431,1,0.709525,0.697582,0.703713,0.691016,0.697576,0.700467,0.709796,0.616540,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1816796431,2,0.711059,0.700858,0.706272,0.693285,0.700825,0.703319,0.711549,0.615606,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1816796431,3,0.712799,0.702518,0.707840,0.694899,0.702445,0.704794,0.712483,0.625044,...,-0.370770,-0.408097,-0.185217,-0.325494,-0.343373,-0.328294,-0.203126,-0.315719,-0.326104,-0.314282
4,1816796431,4,0.712349,0.705451,0.709918,0.696006,0.705180,0.706928,0.712685,0.614356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
selected_sequence = 1816796431

landmark_sequence[["sequence_id", "frame"]+hand_columns].query(f"sequence_id=={selected_sequence} and frame<20").sort_values(by="frame")

,sequence_id,frame,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,...,y_left_hand_11,y_left_hand_12,y_left_hand_13,y_left_hand_14,y_left_hand_15,y_left_hand_16,y_left_hand_17,y_left_hand_18,y_left_hand_19,y_left_hand_20
0,1816796431,0,0.408832,0.519912,0.612159,0.707576,0.797313,0.494709,0.532817,0.553556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1816796431,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1816796431,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1816796431,3,0.398663,0.523662,0.638807,0.744236,0.832567,0.538486,0.564302,0.581011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1816796431,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1816796431,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1816796431,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1816796431,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1816796431,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1816796431,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
selected_sequence = 1848182207

landmark_sequence[["sequence_id", "frame"]+hand_columns].query(f"sequence_id=={selected_sequence} and frame<50").sort_values(by="frame")

,sequence_id,frame,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,...,y_left_hand_11,y_left_hand_12,y_left_hand_13,y_left_hand_14,y_left_hand_15,y_left_hand_16,y_left_hand_17,y_left_hand_18,y_left_hand_19,y_left_hand_20
162398,1848182207,0,0.190891,0.291506,0.350984,0.375855,0.387855,0.236709,0.333707,0.348950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162399,1848182207,1,0.200199,0.295628,0.345924,0.367991,0.376934,0.239542,0.329809,0.335190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162400,1848182207,2,0.195625,0.290283,0.343096,0.364881,0.375408,0.250237,0.334807,0.337097,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162401,1848182207,3,0.205127,0.297120,0.347441,0.365882,0.374909,0.253803,0.335501,0.336627,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162402,1848182207,4,0.202317,0.293080,0.343356,0.363386,0.374829,0.253700,0.334738,0.336904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162403,1848182207,5,0.205964,0.295638,0.345972,0.364003,0.376335,0.254384,0.336546,0.337339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162404,1848182207,6,0.203622,0.294830,0.346028,0.364892,0.374141,0.254537,0.333465,0.335103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162405,1848182207,7,0.210250,0.298820,0.349023,0.366875,0.376709,0.255980,0.337309,0.337721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162406,1848182207,8,0.209648,0.296986,0.349131,0.367421,0.380158,0.262043,0.339937,0.342352,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162407,1848182207,9,0.219782,0.303529,0.354605,0.374978,0.387603,0.272913,0.347508,0.349307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# I could keep those missing values by filling them with previous values

# Or I could drop them.

# I think those are null because the hands are out of frame. Not that the person is holding his hand still.

In [16]:
hand_columns

['x_right_hand_0',
 'x_right_hand_1',
 'x_right_hand_2',
 'x_right_hand_3',
 'x_right_hand_4',
 'x_right_hand_5',
 'x_right_hand_6',
 'x_right_hand_7',
 'x_right_hand_8',
 'x_right_hand_9',
 'x_right_hand_10',
 'x_right_hand_11',
 'x_right_hand_12',
 'x_right_hand_13',
 'x_right_hand_14',
 'x_right_hand_15',
 'x_right_hand_16',
 'x_right_hand_17',
 'x_right_hand_18',
 'x_right_hand_19',
 'x_right_hand_20',
 'y_right_hand_0',
 'y_right_hand_1',
 'y_right_hand_2',
 'y_right_hand_3',
 'y_right_hand_4',
 'y_right_hand_5',
 'y_right_hand_6',
 'y_right_hand_7',
 'y_right_hand_8',
 'y_right_hand_9',
 'y_right_hand_10',
 'y_right_hand_11',
 'y_right_hand_12',
 'y_right_hand_13',
 'y_right_hand_14',
 'y_right_hand_15',
 'y_right_hand_16',
 'y_right_hand_17',
 'y_right_hand_18',
 'y_right_hand_19',
 'y_right_hand_20',
 'x_left_hand_0',
 'x_left_hand_1',
 'x_left_hand_2',
 'x_left_hand_3',
 'x_left_hand_4',
 'x_left_hand_5',
 'x_left_hand_6',
 'x_left_hand_7',
 'x_left_hand_8',
 'x_left_hand_9',


In [17]:
# it is not text it is actually pd.na

print(landmark_sequence[hand_columns].iloc[1]["x_right_hand_0"])
print(pd.isna(landmark_sequence[hand_columns].iloc[1]["x_right_hand_0"]))

nan
True


In [18]:
landmark_sequence.head()[["sequence_id", "frame"]+hand_columns]

,sequence_id,frame,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,...,y_left_hand_11,y_left_hand_12,y_left_hand_13,y_left_hand_14,y_left_hand_15,y_left_hand_16,y_left_hand_17,y_left_hand_18,y_left_hand_19,y_left_hand_20
0,1816796431,0,0.408832,0.519912,0.612159,0.707576,0.797313,0.494709,0.532817,0.553556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1816796431,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1816796431,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1816796431,3,0.398663,0.523662,0.638807,0.744236,0.832567,0.538486,0.564302,0.581011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1816796431,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
landmark_sequence.head()[["sequence_id", "frame"]+hand_columns].dropna(subset=hand_columns, how="all")

,sequence_id,frame,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,...,y_left_hand_11,y_left_hand_12,y_left_hand_13,y_left_hand_14,y_left_hand_15,y_left_hand_16,y_left_hand_17,y_left_hand_18,y_left_hand_19,y_left_hand_20
0,1816796431,0,0.408832,0.519912,0.612159,0.707576,0.797313,0.494709,0.532817,0.553556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1816796431,3,0.398663,0.523662,0.638807,0.744236,0.832567,0.538486,0.564302,0.581011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# I should do both drop when all rows are nan and fill nan to 0 when one hand is missing.
def dropna(dataframe, columns):
    copy_dataframe = dataframe.copy(deep=True)
    return copy_dataframe.dropna(subset=columns, how="all").reset_index(drop=True)

def fillzero(dataframe):
    copy_dataframe = dataframe.copy(deep=True)
    return copy_dataframe.fillna(0)

In [21]:
landmark_sequence_new = dropna(landmark_sequence, hand_columns)
landmark_sequence_new = fillzero(landmark_sequence_new)

In [22]:
landmark_sequence_new.shape

(95057, 1631)

In [23]:
landmark_sequence.head()

,sequence_id,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
0,1816796431,0,0.710588,0.699951,0.705657,0.691768,0.699669,0.701980,0.709724,0.610405,...,-0.245855,-0.269148,-0.129743,-0.251501,-0.278687,-0.266530,-0.152852,-0.257519,-0.275822,-0.266876
1,1816796431,1,0.709525,0.697582,0.703713,0.691016,0.697576,0.700467,0.709796,0.616540,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1816796431,2,0.711059,0.700858,0.706272,0.693285,0.700825,0.703319,0.711549,0.615606,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1816796431,3,0.712799,0.702518,0.707840,0.694899,0.702445,0.704794,0.712483,0.625044,...,-0.370770,-0.408097,-0.185217,-0.325494,-0.343373,-0.328294,-0.203126,-0.315719,-0.326104,-0.314282
4,1816796431,4,0.712349,0.705451,0.709918,0.696006,0.705180,0.706928,0.712685,0.614356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
landmark_sequence_new.head()[["sequence_id", "frame"]+hand_columns].sort_values(by=["sequence_id", "frame"])

,sequence_id,frame,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,...,y_left_hand_11,y_left_hand_12,y_left_hand_13,y_left_hand_14,y_left_hand_15,y_left_hand_16,y_left_hand_17,y_left_hand_18,y_left_hand_19,y_left_hand_20
0,1816796431,0,0.408832,0.519912,0.612159,0.707576,0.797313,0.494709,0.532817,0.553556,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1816796431,3,0.398663,0.523662,0.638807,0.744236,0.832567,0.538486,0.564302,0.581011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1816796431,12,0.419290,0.509726,0.593165,0.685492,0.777913,0.483669,0.510993,0.536410,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1816796431,13,0.398764,0.498118,0.583356,0.677779,0.775966,0.481279,0.491659,0.524974,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1816796431,14,0.420213,0.495650,0.571790,0.659049,0.749740,0.485707,0.475930,0.501727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
landmark_sequence[["sequence_id", "frame"]+hand_columns].sort_values(by=["sequence_id", "frame"])

,sequence_id,frame,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,...,y_left_hand_11,y_left_hand_12,y_left_hand_13,y_left_hand_14,y_left_hand_15,y_left_hand_16,y_left_hand_17,y_left_hand_18,y_left_hand_19,y_left_hand_20
0,1816796431,0,0.408832,0.519912,0.612159,0.707576,0.797313,0.494709,0.532817,0.553556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1816796431,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1816796431,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1816796431,3,0.398663,0.523662,0.638807,0.744236,0.832567,0.538486,0.564302,0.581011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1816796431,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162694,1848182207,296,0.216413,0.274089,0.312841,0.358865,0.403348,0.192464,0.219928,0.254832,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162695,1848182207,297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162696,1848182207,298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162697,1848182207,299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
landmark_sequence_new.reset_index()[["sequence_id", "frame"]+hand_columns].sort_values(by=["sequence_id", "frame"])

,sequence_id,frame,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,...,y_left_hand_11,y_left_hand_12,y_left_hand_13,y_left_hand_14,y_left_hand_15,y_left_hand_16,y_left_hand_17,y_left_hand_18,y_left_hand_19,y_left_hand_20
0,1816796431,0,0.408832,0.519912,0.612159,0.707576,0.797313,0.494709,0.532817,0.553556,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1816796431,3,0.398663,0.523662,0.638807,0.744236,0.832567,0.538486,0.564302,0.581011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1816796431,12,0.419290,0.509726,0.593165,0.685492,0.777913,0.483669,0.510993,0.536410,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1816796431,13,0.398764,0.498118,0.583356,0.677779,0.775966,0.481279,0.491659,0.524974,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1816796431,14,0.420213,0.495650,0.571790,0.659049,0.749740,0.485707,0.475930,0.501727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95052,1848182207,293,0.215248,0.277645,0.331257,0.384949,0.438683,0.228495,0.238761,0.259253,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95053,1848182207,294,0.217072,0.272005,0.315519,0.357841,0.400638,0.247443,0.262433,0.294195,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95054,1848182207,295,0.229776,0.280101,0.316568,0.361121,0.420181,0.203936,0.225522,0.248192,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95055,1848182207,296,0.216413,0.274089,0.312841,0.358865,0.403348,0.192464,0.219928,0.254832,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# so far 

# first: I get the right and left hand x and y coordinates.
# Drop the frames with no hands
# fill to zero when one hand is not in the sceen, this means the hand is not there. I will need to use a mask to ignore that.

In [29]:
# TfRecord

# Set length of frames to 128
FRAME_LEN = 512
FEATURE_COLUMNS = hand_columns

# Create directory to store the new data
if not os.path.exists(TFRECORDS_PATH+os.sep+"train_landmarks"):
    os.mkdir(TFRECORDS_PATH+os.sep+"train_landmarks")
else:
    shutil.rmtree(TFRECORDS_PATH+os.sep+"train_landmarks")
    os.mkdir(TFRECORDS_PATH+os.sep+"train_landmarks")

# Loop through each file_id
for file_id in tqdm(train_df.file_id.unique(), total=len(train_df.file_id.unique())):
    # Parquet file name
    parquet_file = TRAIN_LANDMARKS_PATH+os.sep+f"{file_id}.parquet"

    # Sequences and frames inside the sequence for that file
    file_df = train_df.loc[train_df["file_id"] == file_id]

    # Fetch the parquet file
    df = pd.read_parquet(parquet_file).reset_index()[["sequence_id", "frame"]+FEATURE_COLUMNS]

    df = dropna(df, columns=FEATURE_COLUMNS)
    df = fillzero(df)
    df = df.sort_values(by=["sequence_id", "frame"])

    # print(df.shape)
    
    # File name for the updated data
    tf_file = TFRECORDS_PATH+os.sep+"train_landmarks"+os.sep+f"{file_id}.tfrecord"

    parquet_numpy = df.to_numpy()
    # Initialize the pointer to write the output of 
    # each `for loop` below as a sequence into the file.

    with tf.io.TFRecordWriter(tf_file) as file_writer:
        # Loop through each sequence in file.
        for seq_id, phrase in zip(file_df.sequence_id, file_df.phrase):
            # Fetch sequence data
            frames_df = df[df.sequence_id == seq_id].sort_values(by=["frame"])

            # to maintain order
            frames_numpy = frames_df[FEATURE_COLUMNS].to_numpy()

            #features dictionary
            # features = {FEATURE_COLUMNS[i]: tf.train.Feature(
            #     float_list=tf.train.FloatList(value=frames[:, i])) for i in range(len(FEATURE_COLUMNS))}

            #one feature all values from that sequence in order.
            features = {FEATURE_COLUMNS[i]: tf.train.Feature(
                float_list=tf.train.FloatList(value=frames_numpy[:, i])) for i in range(len(FEATURE_COLUMNS))}            

            # print(features)
            # break

            features["phrase"] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(phrase, 'utf-8')]))

            record_bytes = tf.train.Example(features=tf.train.Features(feature=features)).SerializeToString()

            file_writer.write(record_bytes)
    # break

  0%|          | 0/68 [00:00<?, ?it/s]

In [30]:
list_files = np.array([TFRECORDS_PATH+os.sep+"train_landmarks"+os.sep+file_name for file_name in os.listdir(TFRECORDS_PATH+os.sep+"train_landmarks/")])

In [31]:
list_files[:5]

array(['../data/asl-fingerspelling/tfrecords/train_landmarks/2026717426.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/1726141437.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/1664666588.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/638508439.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/1497621680.tfrecord'],
      dtype='<U72')

In [32]:
import math

N = list_files.shape[0]*512

train_split = math.ceil(N*0.8)
validation_split = N - train_split

number_of_train_files = math.ceil(train_split/512)

np.random.shuffle(list_files)

train_files = list_files[:number_of_train_files]
val_files = list_files[number_of_train_files:]

print(f"train split: {train_files.shape[0]*512} | val split: {val_files.shape[0]*512}")

train split: 28160 | val split: 6656


In [33]:
train_files[:5]

array(['../data/asl-fingerspelling/tfrecords/train_landmarks/1557244878.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/2026717426.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/1021040628.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/568753759.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/2072296290.tfrecord'],
      dtype='<U72')

In [34]:
train_dataset = tf.data.TFRecordDataset(train_files[0])

2023-08-14 21:15:45.540273: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-14 21:15:45.612943: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-14 21:15:45.613128: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [35]:
next(iter(train_dataset))

<tf.Tensor: shape=(), dtype=string, numpy=b'\n\xd8b\n\x94\x01\n\x0fy_right_hand_13\x12\x80\x01\x12~\n|On<?\xf7\xf3)?\x9dk,?\x0e<#?\xc9`!?i\x9b(?|\xcf ?\x95\x19\x1e?\x1ec\x1d?\xa7\x90\x1e?\xec\xd9\x1e?A\xcd\x1e?\xe1\xbf\x1f?\x97\xa5\x1f?\x12%\x1f?\xafc ?\xfa{\x1f?(\x94 ?\xf8(\x1e?\x89\xba\x1d?]\xb9\x1e?\xd0\x8f\x1e?J@ ?,i ?\x12\xea)?Q\xea(?\xe8\n7?\xf7\x98+?\xc5u0?\xfe\xa5.?F\xc72?\n\x93\x01\n\x0ex_right_hand_8\x12\x80\x01\x12~\n|(\x8e\xa6>>\x9a\xb2>+p\x87>R\xefm>\xa6\x94\x82>\x08\xd5\xb9>H\xc4\xb5>\x94v\xb2>\x05K\xb2><\xac\xaf>\xe9\xc4\xb1>\x01\'\xb6>\xb1\xce\xbc>&$\xb9>0 \xbe>\x1e\xee\xc2>t\xe4\xc3>B\xba\xba>\xb2\xfa\xcb>4k\xcb>\x95*\xcc>\\\xcb\xcd>\xa6U\xc1>$r\xce>\x84\xd4\xb1>\x85\xdf\xba>\x7fU\x93>\x94\xe3\xd8>\xbb\xc0\xef>\xaeI\xbb>\xbc\xff\xcc>\n\x93\x01\n\x0ex_left_hand_12\x12\x80\x01\x12~\n|\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [36]:
def decode_fn(record_bytes):
    schema = {COL: tf.io.VarLenFeature(dtype=tf.float32) for COL in FEATURE_COLUMNS}
    schema["phrase"] = tf.io.FixedLenFeature([], dtype=tf.string)
    features = tf.io.parse_single_example(record_bytes, schema)
    phrase = features["phrase"]
    landmarks = ([tf.sparse.to_dense(features[COL]) for COL in FEATURE_COLUMNS])
    # Transpose to maintain the original shape of landmarks data.
    landmarks = tf.transpose(landmarks)
    
    return landmarks, phrase

In [47]:
train_dataset = tf.data.TFRecordDataset(train_files).map(decode_fn)

In [50]:
FEATURE_COLUMNS

['x_right_hand_0',
 'x_right_hand_1',
 'x_right_hand_2',
 'x_right_hand_3',
 'x_right_hand_4',
 'x_right_hand_5',
 'x_right_hand_6',
 'x_right_hand_7',
 'x_right_hand_8',
 'x_right_hand_9',
 'x_right_hand_10',
 'x_right_hand_11',
 'x_right_hand_12',
 'x_right_hand_13',
 'x_right_hand_14',
 'x_right_hand_15',
 'x_right_hand_16',
 'x_right_hand_17',
 'x_right_hand_18',
 'x_right_hand_19',
 'x_right_hand_20',
 'y_right_hand_0',
 'y_right_hand_1',
 'y_right_hand_2',
 'y_right_hand_3',
 'y_right_hand_4',
 'y_right_hand_5',
 'y_right_hand_6',
 'y_right_hand_7',
 'y_right_hand_8',
 'y_right_hand_9',
 'y_right_hand_10',
 'y_right_hand_11',
 'y_right_hand_12',
 'y_right_hand_13',
 'y_right_hand_14',
 'y_right_hand_15',
 'y_right_hand_16',
 'y_right_hand_17',
 'y_right_hand_18',
 'y_right_hand_19',
 'y_right_hand_20',
 'x_left_hand_0',
 'x_left_hand_1',
 'x_left_hand_2',
 'x_left_hand_3',
 'x_left_hand_4',
 'x_left_hand_5',
 'x_left_hand_6',
 'x_left_hand_7',
 'x_left_hand_8',
 'x_left_hand_9',


In [49]:
len(FEATURE_COLUMNS)

84

In [52]:
next(iter(train_dataset))[0][0]

<tf.Tensor: shape=(84,), dtype=float32, numpy=
array([0.14810829, 0.24183479, 0.32340184, 0.36693922, 0.39620632,
       0.28303185, 0.3136069 , 0.32453594, 0.32530332, 0.2130632 ,
       0.25527725, 0.27420315, 0.27943808, 0.14105727, 0.18323986,
       0.21422733, 0.23094925, 0.06893495, 0.06349775, 0.07358548,
       0.09314722, 0.8668769 , 0.8444395 , 0.7948865 , 0.75446016,
       0.7230924 , 0.7295317 , 0.70797724, 0.7576676 , 0.7966458 ,
       0.72863126, 0.7163163 , 0.77389103, 0.8146938 , 0.7360582 ,
       0.71572226, 0.76461226, 0.80396295, 0.74967355, 0.6936214 ,
       0.6569228 , 0.6255688 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.      

In [53]:
train_df.head()

,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road


In [60]:
TRAIN_LANDMARKS_PATH+os.sep+f"{5414471}.tfrecord"

'../data/asl-fingerspelling/train_landmarks/5414471.tfrecord'

In [67]:
train_files[0]

'../data/asl-fingerspelling/tfrecords/train_landmarks/1557244878.tfrecord'

In [68]:
parquet = pd.read_parquet(TRAIN_LANDMARKS_PATH+os.sep+f"{1557244878}.parquet").reset_index()

In [69]:
parquet.head()[FEATURE_COLUMNS]

,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,x_right_hand_8,x_right_hand_9,...,y_left_hand_11,y_left_hand_12,y_left_hand_13,y_left_hand_14,y_left_hand_15,y_left_hand_16,y_left_hand_17,y_left_hand_18,y_left_hand_19,y_left_hand_20
0,0.148108,0.241835,0.323402,0.366939,0.396206,0.283032,0.313607,0.324536,0.325303,0.213063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# it seems to be fine but it does not allow me to batch the tfrecords.